## Purse-ETL Quick Start

## Extract Futures OHLCV from Binance Data Collection

In [1]:
from etl.extractor.binance_collector import BinanceFuturesUM
from etl.extractor.binance_collector.types import TimeFrame

# Binance Data Collection Extractor
extractor_um = BinanceFuturesUM(ticker='BTCUSDT', timeframe=TimeFrame.MINUTE5)

# Klines Data
klines = extractor_um.load(start_date='2024-09-12', end_date='2024-09-12')

2024-09-18 02:26:19,108 - INFO - Files in ZIP for 2024-09-12: ['BTCUSDT-5m-2024-09-12.csv']
2024-09-18 02:26:19,120 - INFO - Loaded data with shape: (288, 12)


## OHLCV Data Transformer

In [2]:
from etl.transformer.binance_collector import BinanceKlinesTransformer

transformer = BinanceKlinesTransformer()
ohlcv = transformer.transform(klines)
ohlcv

,open,high,low,close,volume
datetime,,,,,
2024-09-12 00:00:00,57305.6,57384.6,57283.1,57343.7,657.984
2024-09-12 00:05:00,57343.8,57425.4,57343.8,57421.9,416.098
2024-09-12 00:10:00,57421.9,57489.9,57378.0,57424.2,809.946
2024-09-12 00:15:00,57424.2,57483.9,57386.2,57465.4,510.603
2024-09-12 00:20:00,57465.4,57694.0,57439.9,57627.3,2332.724
...,...,...,...,...,...
2024-09-12 23:35:00,58058.9,58122.0,58058.9,58110.1,213.390
2024-09-12 23:40:00,58110.1,58158.0,58084.8,58085.5,218.786
2024-09-12 23:45:00,58085.5,58139.5,58070.0,58132.2,335.772


## Data Load to MySQL DB

In [4]:
from etl.loader import SQLDatabaseLoader
from etl.utils import load_db_config 
from etl.utils import get_db_uri 

# Get DB Config
db_config = load_db_config()

# Get DB URI
db_uri = get_db_uri(db_config)

# Loader
loader = SQLDatabaseLoader(db_uri=db_uri)
loader.connect

<bound method SQLDatabaseLoader.connect of <etl.loader.sql_loader.SQLDatabaseLoader object at 0x7fbb0a217f40>>

In [10]:
from etl.loader.table import OHLCV
from etl.loader.query import upsert_query_func

ticker = 'BTCUSDt'

# OHLCV Object
table = OHLCV(
    datetime=ohlcv.index.tolist(),
    open=ohlcv.open.tolist(),
    high=ohlcv.high.tolist(),
    low=ohlcv.low.tolist(),
    close=ohlcv.close.tolist(),
    volume=ohlcv.volume.tolist(),
)

# Prefixed Upsert Query
query = upsert_query_func(
    table, table_name=f"ohlcv_{ticker}".lower(), unique_key="datetime"
)

In [ ]:
# Data Transaction 

loader.transaction(query)
loader.close()